In [ ]:
from collections import defaultdict

class NFA:
    #five tuble that used to represent the FA
    def __init__(self, states, alphabet, transitions, start_state, accept_states):
        self.states = states
        self.alphabet = alphabet
        self.transitions = transitions  # dictionary where key: (state, symbol) -> value: set of states
        self.start_state = start_state
        self.accept_states = accept_states

    def epsilon_closure(self, states):
        """ Calculate the epsilon closure for a set of states """
        stack = list(states)
        closure = set(states)
        
        while stack:
            state = stack.pop()
            for next_state in self.transitions.get((state, ''), set()):
                if next_state not in closure:
                    closure.add(next_state)
                    stack.append(next_state)
        
        return closure

class DFA:
    def __init__(self, nfa):
        self.nfa = nfa
        self.dfa_states = {}
        self.dfa_transitions = {}
        self.start_state = frozenset(nfa.epsilon_closure({nfa.start_state}))
        self.accept_states = set()
        self.create_dfa()

    def create_dfa(self):
        unmarked_states = [self.start_state]
        self.dfa_states[self.start_state] = True  # Marked as processed

        while unmarked_states:
            current_dfa_state = unmarked_states.pop()
            for symbol in self.nfa.alphabet:
                if symbol == '': continue  # Skip epsilon transitions
                
                next_nfa_states = set()
                for nfa_state in current_dfa_state:
                    next_nfa_states.update(self.nfa.transitions.get((nfa_state, symbol), set()))
                
                # Apply epsilon closure to the result of the move
                next_dfa_state = frozenset(self.nfa.epsilon_closure(next_nfa_states))

                if next_dfa_state not in self.dfa_states:
                    unmarked_states.append(next_dfa_state)
                    self.dfa_states[next_dfa_state] = True

                    # Check if this DFA state is an accept state
                    if any(state in self.nfa.accept_states for state in next_dfa_state):
                        self.accept_states.add(next_dfa_state)

                # Add the transition
                self.dfa_transitions[(current_dfa_state, symbol)] = next_dfa_state

    def display_dfa(self):
        print("DFA States:")
        for state in self.dfa_states:
            print(f"  {state}")
        print("\nDFA Transitions:")
        for (state, symbol), next_state in self.dfa_transitions.items():
            print(f"  {state} --{symbol}--> {next_state}")
        print("\nDFA Start State:", self.start_state)
        print("DFA Accept States:", self.accept_states)

# Define the NFA
states = {'q0', 'q1', 'q2','q4'}
alphabet = {'a', 'b'}
transitions = {
    ('q0', 'a'): {'q0', 'q1'},
    ('q0', 'b'): {'q0'},
    ('q1', 'a'): {'q2'},
    ('q1','b') : {'q1'},
    ('q2','a'):{'q3'},
    ('q2','b'):{'q3'},
    ('q3','b'):{'q2'}

}
start_state = 'q0'
accept_states = {'q3','q2'}

# Create NFA and convert to DFA
nfa = NFA(states, alphabet, transitions, start_state, accept_states)
dfa = DFA(nfa)

# Display DFA
dfa.display_dfa()
